# 📊 Notebook 01: Data Collection

**Predicting F1 Race Finishing Positions Using Practice & Qualifying Data**

This notebook collects all F1 session data for 2023, 2024, and 2025 seasons using the **FastF1** library.

- Practice sessions (FP1, FP2, FP3)
- Qualifying results
- Race results (target variable)
- Weather conditions

All data is cached locally and saved as Parquet files for downstream use.

In [1]:
# Standard imports
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import numpy as np
import fastf1

import config
from src.data_collection import (
    get_race_schedule,
    collect_season_data,
    collect_and_save_all_data,
    load_saved_data,
)

print(f"FastF1 version: {fastf1.__version__}")
print(f"Cache dir: {config.CACHE_DIR}")
print(f"Processed dir: {config.PROCESSED_DIR}")

FastF1 version: 3.8.1
Cache dir: /Users/t.joongpiriyapong/Documents MacHDD/Workspace/f1-race-data-analytics/data/cache
Processed dir: /Users/t.joongpiriyapong/Documents MacHDD/Workspace/f1-race-data-analytics/data/processed


## 1. Race Schedule Overview

Let's first inspect the race calendar for each season.

In [2]:
# Show race schedules for all years
for year in config.ALL_YEARS:
    schedule = get_race_schedule(year)
    print(f"\n{'='*50}")
    print(f"{year} Season: {len(schedule)} races")
    print(f"{'='*50}")
    display(schedule[['RoundNumber', 'EventName', 'Location', 'EventDate']].head(25))


2022 Season: 22 races


,RoundNumber,EventName,Location,EventDate
0,1,Bahrain Grand Prix,Sakhir,2022-03-20
1,2,Saudi Arabian Grand Prix,Jeddah,2022-03-27
2,3,Australian Grand Prix,Melbourne,2022-04-10
3,4,Emilia Romagna Grand Prix,Imola,2022-04-24
4,5,Miami Grand Prix,Miami,2022-05-08
5,6,Spanish Grand Prix,Barcelona,2022-05-22
6,7,Monaco Grand Prix,Monaco,2022-05-29
7,8,Azerbaijan Grand Prix,Baku,2022-06-12
8,9,Canadian Grand Prix,Montréal,2022-06-19
9,10,British Grand Prix,Silverstone,2022-07-03



2023 Season: 22 races


,RoundNumber,EventName,Location,EventDate
0,1,Bahrain Grand Prix,Sakhir,2023-03-05
1,2,Saudi Arabian Grand Prix,Jeddah,2023-03-19
2,3,Australian Grand Prix,Melbourne,2023-04-02
3,4,Azerbaijan Grand Prix,Baku,2023-04-30
4,5,Miami Grand Prix,Miami,2023-05-07
5,6,Monaco Grand Prix,Monaco,2023-05-28
6,7,Spanish Grand Prix,Barcelona,2023-06-04
7,8,Canadian Grand Prix,Montréal,2023-06-18
8,9,Austrian Grand Prix,Spielberg,2023-07-02
9,10,British Grand Prix,Silverstone,2023-07-09



2024 Season: 24 races


,RoundNumber,EventName,Location,EventDate
0,1,Bahrain Grand Prix,Sakhir,2024-03-02
1,2,Saudi Arabian Grand Prix,Jeddah,2024-03-09
2,3,Australian Grand Prix,Melbourne,2024-03-24
3,4,Japanese Grand Prix,Suzuka,2024-04-07
4,5,Chinese Grand Prix,Shanghai,2024-04-21
5,6,Miami Grand Prix,Miami,2024-05-05
6,7,Emilia Romagna Grand Prix,Imola,2024-05-19
7,8,Monaco Grand Prix,Monaco,2024-05-26
8,9,Canadian Grand Prix,Montréal,2024-06-09
9,10,Spanish Grand Prix,Barcelona,2024-06-23



2025 Season: 24 races


,RoundNumber,EventName,Location,EventDate
0,1,Australian Grand Prix,Melbourne,2025-03-16
1,2,Chinese Grand Prix,Shanghai,2025-03-23
2,3,Japanese Grand Prix,Suzuka,2025-04-06
3,4,Bahrain Grand Prix,Sakhir,2025-04-13
4,5,Saudi Arabian Grand Prix,Jeddah,2025-04-20
5,6,Miami Grand Prix,Miami Gardens,2025-05-04
6,7,Emilia Romagna Grand Prix,Imola,2025-05-18
7,8,Monaco Grand Prix,Monaco,2025-05-25
8,9,Spanish Grand Prix,Barcelona,2025-06-01
9,10,Canadian Grand Prix,Montréal,2025-06-15


## 2. Collect All Season Data

This step fetches all practice, qualifying, and race data for 2023, 2024, and 2025.

⚠️ **First run will take 30-60 minutes** as data is downloaded from FastF1 servers.  
Subsequent runs use the local cache and are much faster.

In [3]:
# Collect and save all data
# This is the main data collection step
collect_and_save_all_data()

core           INFO 	Loading data for Bahrain Grand Prix - Practice 1 [v3.8.1]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...




  Round 1: Bahrain Grand Prix...


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

    ✓ Practice 1: 387 laps


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

...

    ✓ Race: 20 classified
    ✓ Weather: track=28.6°C

  Round 2: Saudi Arabian Grand Prix...


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

    ✓ Practice 1: 439 laps


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...


KeyboardInterrupt: 

## 3. Verify Collected Data

Load the saved data and inspect its quality.

In [7]:
# Verify saved data for each year
for year in config.ALL_YEARS:
    print(f"\n{'='*60}")
    print(f"Verifying {year} data")
    print(f"{'='*60}")
    data = load_saved_data(year)
    
    for key, df in data.items():
        if not df.empty:
            print(f"\n  {key}: {df.shape[0]} rows × {df.shape[1]} columns")
            print(f"    Columns: {list(df.columns)}")
        else:
            print(f"\n  {key}: EMPTY")


Verifying 2022 data

  practice_laps: 10032 rows × 26 columns
    Columns: ['DriverNumber', 'Driver', 'Team', 'LapNumber', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'Compound', 'TyreLife', 'Stint', 'IsPersonalBest', 'FreshTyre', 'TrackStatus', 'LapTime_sec', 'Sector1Time_sec', 'Sector2Time_sec', 'Sector3Time_sec', 'Year', 'RoundNumber', 'EventName', 'SessionType', 'LapTime_seconds', 'Sector1Time_seconds', 'Sector2Time_seconds', 'Sector3Time_seconds']

  qualifying: 160 rows × 15 columns
    Columns: ['DriverNumber', 'Driver', 'Team', 'quali_position', 'Q1_sec', 'Q2_sec', 'Q3_sec', 'quali_best_time', 'quali_gap_to_pole', 'Year', 'RoundNumber', 'EventName', 'Q1_seconds', 'Q2_seconds', 'Q3_seconds']

  race_results: 160 rows × 13 columns
    Columns: ['DriverNumber', 'Driver', 'Team', 'grid_position', 'race_position', 'ClassifiedPosition', 'Status', 'Points', 'Year', 'RoundNumber', 'EventName', 'circuit_key', 'is_street_circuit']

  weather: 8 rows × 8 columns
    Columns: ['track_temp

In [ ]:
# Quick look at the 2023 race results
data_2023 = load_saved_data(2023)
print("\n── 2023 Race Results (sample) ──")
display(data_2023['race_results'].head(20))

In [ ]:
# Quick look at practice lap data
print("\n── 2023 Practice Laps (sample) ──")
display(data_2023['practice_laps'].head(10))

print(f"\nLaps per session type:")
print(data_2023['practice_laps'].groupby('SessionType').size())

In [ ]:
# Qualifying data
print("\n── 2023 Qualifying Data (sample) ──")
display(data_2023['qualifying'].head(10))

In [ ]:
# Weather data
print("\n── 2023 Weather Data ──")
display(data_2023['weather'])

## 4. Data Summary Statistics

Overview of what we collected for downstream analysis.

In [8]:
# Summary across all years
summary_rows = []
for year in config.ALL_YEARS:
    data = load_saved_data(year)
    row = {
        'Year': year,
        'Practice Laps': len(data['practice_laps']),
        'Qualifying Entries': len(data['qualifying']),
        'Race Results': len(data['race_results']),
        'Weather Records': len(data['weather']),
        'Races': data['race_results']['RoundNumber'].nunique() if not data['race_results'].empty else 0,
        'Drivers': data['race_results']['Driver'].nunique() if 'Driver' in data['race_results'].columns else 0,
    }
    summary_rows.append(row)

summary_df = pd.DataFrame(summary_rows)
print("\n📊 DATA COLLECTION SUMMARY")
print("=" * 60)
display(summary_df)

print(f"\n✅ Data collection complete!")
print(f"   Training: {config.TRAIN_YEARS} → {summary_df[summary_df['Year'].isin(config.TRAIN_YEARS)]['Race Results'].sum()} samples")
print(f"   Testing:  {config.TEST_YEARS} → {summary_df[summary_df['Year'].isin(config.TEST_YEARS)]['Race Results'].sum()} samples")


📊 DATA COLLECTION SUMMARY


,Year,Practice Laps,Qualifying Entries,Race Results,Weather Records,Races,Drivers
0,2022,10032,160,160,8,8,21
1,2023,23474,440,440,22,22,22
2,2024,26076,479,479,24,24,24
3,2025,27986,480,479,24,24,21



✅ Data collection complete!
   Training: [2022, 2023, 2024] → 1079 samples
   Testing:  [2025] → 479 samples
